# artm.LDA class usage (BigARTM Python API).

Author - **Murat Apishev** (great-mel@yandex.ru)

artm.LDA was designed for non-advanced users with minimal knowledge about topic modeling and ARTM. It is a cutted version of artm.ARTM model with pre-defined scores and regularizers. artm.LDA has enough abilities for fitting the LDA with regularizers of smoothing/sparsing of $\Phi$ and $\Theta$ matrices with offline or online algorithms. Also it can compute scores of perplexity, matrices sparsities and most probable tokens in each topic, and return the whole resulting matrices.

You cna find the information about artm.LDA here: http://bigartm.readthedocs.org/en/master/python_interface/lda.html

Let's simple model experiment with kos collection in UCI format. You can read about BatchVectorizer and dictionaries in the introduction of this paper: http://nbviewer.ipython.org/github/bigartm/bigartm-book/blob/master/ARTM_tutorial_RU.ipynb

The collection can be downloaded from here: https://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/

In link above you also can read about the UCI Bag-Of-Words format. You will need two files: docword.kos.txt и vocab.kos.txt.
Both these files should be put into the same directory with this notebook.

Let's import the artm module, create BatchVectorizer and run dictionary gathering inside of it (if you are interested in the detailes, you need to read information from given links):

### 1 . The most basic part.

In [1]:
import artm

print artm.version()

0.8.0


In [ ]:
batch_vectorizer = artm.BatchVectorizer(data_path='.', data_format='bow_uci',
                                        collection_name='kos', target_folder='kos_batches')

Now let's create the model by defining the number of topics, number of passes through each document, hyperparameters of smoothing of $\Phi$ and $\Theta$ and dictionary to use. Also let's ask thr model to store $\Theta$ matrix to have an ability to look at it in the future.

Also you can set here the num_processors parameter, which defines the number of threads to be used on your machine for parallelizing the computing:

In [ ]:
lda = artm.LDA(num_topics=15, alpha=0.01, beta=0.001,
               num_document_passes=5, dictionary=batch_vectorizer.dictionary,
               cache_theta=True)

Well, let's run the learning process using offline algorithm:

In [ ]:
lda.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=10)

That's all, the fitting is over. You can iterate this process, edit model parameters etc.

Now you are able to look at the results of modeling. For instance, let's look at final values of matrices sparsities:

In [ ]:
lda.sparsity_phi_last_value
lda.sparsity_theta_last_value

Or at all values of perplexity (e.g. on each collection pass):

In [ ]:
lda.perplexity_value

You can see the most probable tokens in each topic, at last. The are returned as list of lists of strings (each internal list correspondes to on topic by order). Let's output them with pre-formatting:

In [ ]:
top_tokens = lda.get_top_tokens(num_tokens=10)
for i, token_list in enumerate(top_tokens):
    print 'Topic #{0}: {1}'.format(i, token_list)

To get the matrices you can use th following calls:

In [ ]:
phi = lda.phi_
theta = lda.get_theta()

### 2. Some more detailes.

Here's two more abilities of artm.LDA.

At first, it is the ability to create $\Theta$ matrix for new documents after the model was fitted:

In [ ]:
batch_vectorizer = artm.BatchVectorizer(data_path='kos_batches_test')
theta_test = lda.transform(batch_vectorizer=test_batch_vectorizer)

Secondly, in the case, when you need a custom regularization of each topic in $\Phi$ matrix, you need to set beta a list instead of scalar value. The list should have th length equal to the number of topics, and then each topic will be regularized with corresponding coefficient:

In [ ]:
beta = [0.1] * num_topics  # change as you need
lda = artm.LDA(num_topics=15, alpha=0.01, beta=beta, num_document_passes=5, dictionary=dictionary, cache_theta=True)

### Conclusions

Describde model is the simpliest way to study topic model in BigARTM. You need nothing except BatchVectorizer usage and running couple of code strings described above.

If you need more advanced abilities of the library, you should use artm.ARTM model. Description of this model is given in
main tutorial.